In [1]:
from imutils.video import VideoStream
import numpy as np

import cv2
import imutils
import time

from imutils.video import FileVideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2

path_vid = 'Material/Videos/full.mp4'
path_vid_small = 'Material/Videos/short.mp4'

In [7]:
# video = VideoStream(src=path_vid).start()
 
# # allow the camera or video file to warm up
# time.sleep(2.0)
# initial = True
# flag = False
# current_key_pressed = set()
# circle_radius = 30

In [2]:
fvs = FileVideoStream(path_vid).start()
time.sleep(1.0)

# while True:
#     frame = fvs.read()
    
#     if frame is None:
#         break
        
#     frame = imutils.resize(frame, width=450)
#     cv2.imshow('Vid', frame)
     
#     if cv2.waitKey(1) == 13: #13 is the Enter Key
#         break
    
cv2.destroyAllWindows()

In [23]:
cv2.destroyAllWindows()
fvs.stop()

In [3]:
def draw(ret, bbox, frame):
         # Draw bounding box
    if ret:
        # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
        # Tracking failure
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)


In [4]:
import ctypes
import time

SendInput = ctypes.windll.user32.SendInput


W = 0x11
A = 0x1E
S = 0x1F
D = 0x20

# C struct redefinitions 
PUL = ctypes.POINTER(ctypes.c_ulong)
class KeyBdInput(ctypes.Structure):
    _fields_ = [("wVk", ctypes.c_ushort),
                ("wScan", ctypes.c_ushort),
                ("dwFlags", ctypes.c_ulong),
                ("time", ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class HardwareInput(ctypes.Structure):
    _fields_ = [("uMsg", ctypes.c_ulong),
                ("wParamL", ctypes.c_short),
                ("wParamH", ctypes.c_ushort)]

class MouseInput(ctypes.Structure):
    _fields_ = [("dx", ctypes.c_long),
                ("dy", ctypes.c_long),
                ("mouseData", ctypes.c_ulong),
                ("dwFlags", ctypes.c_ulong),
                ("time",ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class Input_I(ctypes.Union):
    _fields_ = [("ki", KeyBdInput),
                 ("mi", MouseInput),
                 ("hi", HardwareInput)]

class Input(ctypes.Structure):
    _fields_ = [("type", ctypes.c_ulong),
                ("ii", Input_I)]

# Actuals Functions

def PressKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

def ReleaseKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008 | 0x0002, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

if __name__ == '__main__':
    PressKey(0x11)
    time.sleep(1)
    ReleaseKey(0x11)
    time.sleep(1)

In [5]:
def get_centroid(bbox):
    x, y, w, h = bbox
    centx = int(x + w // 2)
    centy = int(y + h // 2)
    return (centx, centy)



def drawbox(bbox, frame):
    global q
    
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
#     print(p1)
    if p1[0] < 10:
#         while True:
        print(bbox)
        q= True #13 is the Enter Key
#             break

In [6]:
import threading 

def gv():
    img_res = requests.get(url+'/shot.jpg')
    img_arr = np.array(bytearray(img_res.content), dtype = np.uint8)
    img = cv2.imdecode(img_arr,-1)
    img = cv2.flip(img, 1)
#     img = imutils.resize(img, width=450)
    return img

In [45]:
def action_key(key):
    PressKey(key)
    pass

pressed_s, pressed_a = None, None
delay = 0.3
last = 0 #last extreme of steer
chng = 3 #change in last, to remove small changes fulucations
chngt = 3 #to restart change to 0, if moved towards centre by some value
prev = 0 #store last pos of steer, so that when returngin towards centre, doesnt press that key

RADIUS = 60
THRESH_ACT = RADIUS#
TAS = 40 #ta steer

cur = 0
per = 2

def steer(bbox):
    #left
    global pressed_s, last, prev, cur, per
    x, _ = get_centroid(bbox)
    diff = x - centleft[0]
    
    if abs(diff) < last: #2ns so, new diff is more, 
        if abs(diff) < last-chng*chngt:
            last = 0
        return
#     prev = diff
    last = abs(diff) + chng
    
    if abs(diff) < TAS:
            return
#     cur +=1
#     if not cur%per==0:
#         return
    if diff > 0:
        pressed_s = D
    else:
        pressed_s = A
        
    action_key(pressed_s)
    return diff    
    
def accelerate(bbox):
    global pressed_a
    
    _, y = get_centroid(bbox)
    diff = - (y - centright[1])  #as y decr upward

    if abs(diff) < THRESH_ACT:
        return
    
    if diff > 0:
        pressed_a = W
    else:
        pressed_a = S
        
    action_key(pressed_a)
    
    return diff


def drawc(frame, center, radius=RADIUS, color=COLOR_BLACK, thickness=3):
    cv2.circle(frame, center, radius, color, thickness)
    
# def release():
   
    

def action(bboxleft, bboxright):
    global pressed_s, pressed_a
    
    a=accelerate(bboxright)
    s=steer(bboxleft)
#     if a is None:
    if s is None and pressed_s is not None:
        ReleaseKey(pressed_s)
        pressed_s = None
    if a is None and pressed_a is not None:
        ReleaseKey(pressed_a)
        pressed_a = None
    return a, s

def releaseAll():
    for key in [W,S,A,D]:
        ReleaseKey(key)

In [52]:

bboxleft, bboxright = bbl, bbr #######

trackerleft = cv2.TrackerCSRT_create()
trackerright = cv2.TrackerCSRT_create()

# Initialize tracker with first frame and bounding box
trackerleft.init(FRAME, bboxleft)
trackerright.init(FRAME, bboxright)


###########################


cv2.putText(frame, 'put both your hands', (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);

for t in range(5*mult, 0, -1):
    frame = gv()
    cv2.putText(frame, str(t//mult), (225,255), cv2.FONT_HERSHEY_SIMPLEX, 1.5, COLOR_RED, 4);
    drawbox(bboxleft, frame)
    drawbox(bboxright, frame)
    cv2.imshow("Tracking", frame)
    cv2.waitKey(1)

cv2.destroyAllWindows()
# a=0
tim = 150
t = time.time()

q = False #to quit on top corener

while True:
#     a+=1
#     if a==15:
#         break
    frame = gv()
    
    if frame is None or time.time()-t>tim:
        break
        
#     frame = cv2.flip(frame, 1)
#     frame = imutils.resize(frame, width=size_width)  

    retleft, bboxleft = trackerleft.update(frame)
    retright, bboxright = trackerright.update(frame)
    
    if not retleft == retright == True:
        print("Tracking stopped")
        break
    
    a,s  = action(bboxleft, bboxright)
    
    cv2.putText(frame, "a  "+str(a), (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
    cv2.putText(frame, "s  "+str(s), (100,90), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
     
    drawbox(bboxleft, frame)
    drawbox(bboxright, frame)

    drawc(frame, centleft)
    drawc(frame, centright)

#     write('')

    cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK,2);
#     cv2.putText(frame, "FPSd : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
    cv2.imshow("Tracking", frame)
    
    k = cv2.waitKey(1)
    
#     if k == 'z':
#         while True:
#             if cv2.waitKey(1) == 'z': #13 is the Enter Key
#                 break
                
    if k == 13 or q: #13 is the Enter Key
        break
    
releaseAll()
    
cv2.destroyAllWindows()
# fvs.stop()

(529.7866821289062, -12.366003036499023, 77.51995086669922, 71.39995574951172)


(466, 174, 73, 64)

In [19]:
cv2.destroyAllWindows()

In [42]:
import threading 
import requests

tracker_type = 'CSRT'
url = 'http://192.168.43.1:8080'

def gv():
    img_res = requests.get(url+'/shot.jpg')
    img_arr = np.array(bytearray(img_res.content), dtype = np.uint8)
    img = cv2.imdecode(img_arr,-1)
    img = cv2.flip(img, 1)
    img = imutils.resize(img, width=640)
    return img

TIMER = 3
mult = 10


for t in range(TIMER*mult, 0, -1):
    frame = gv()
    cv2.putText(frame, str(t//mult), (225,255), cv2.FONT_HERSHEY_SIMPLEX, 1.5, COLOR_RED, 4);
    cv2.imshow("Tracking", frame)
    cv2.waitKey(1)

# fvs = FileVideoStream(path_vid).start()
# frame = gv()
# frame = cv2.flip(frame, 1)

cv2.destroyAllWindows()

trackerleft = cv2.TrackerCSRT_create()
trackerright = cv2.TrackerCSRT_create()


cv2.putText(frame, 'left select', (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
bboxleft = cv2.selectROI(frame.copy(), False)

cv2.putText(frame, 'right select', (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
bboxright = cv2.selectROI(frame.copy(), False)

centleft = get_centroid(bboxleft)
centright = get_centroid(bboxright)

drawc(frame, centleft)
drawc(frame, centright)

# Initialize tracker with first frame and bounding box
trackerleft.init(frame, bboxleft)
trackerright.init(frame, bboxright)

cv2.destroyAllWindows()

bbl, bbr = bboxleft, bboxright
FRAME = frame.copy()

In [ ]:
cv2.destroyAllWindows()

In [35]:
# while True:
#     if cv2.waitKey(1) == 13: #13 is the Enter Key
#         break

(213.95008850097656, 152.9199676513672, 61.91986846923828, 63.593379974365234)

In [297]:
# bboxleft, bboxright = bbl, bbr 

# cv2.putText(frame, 'put both your hands', (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);

# for t in range(4*mult, 0, -1):
#     frame = gv()
#     cv2.putText(frame, str(t//mult), (225,255), cv2.FONT_HERSHEY_SIMPLEX, 1.5, COLOR_RED, 4);
#     drawbox(bboxleft, frame)
#     drawbox(bboxright, frame)
#     cv2.imshow("Tracking", frame)
#     cv2.waitKey(1)

# cv2.destroyAllWindows()

# while True:
#     frame = gv()
    
#     if frame is None:
#         break
        
# #     frame = cv2.flip(frame, 1)
# #     frame = imutils.resize(frame, width=size_width)  

#     retleft, bboxleft = trackerleft.update(frame)
#     retright, bboxright = trackerright.update(frame)
    
#     if not retleft == retright == True:
#         print("Tracking stopped")
#         break
    
#     a,s  = action(bboxleft, bboxright)
    
#     cv2.putText(frame, "a  "+str(a), (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
#     cv2.putText(frame, "s  "+str(s), (100,90), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
     
#     drawbox(bboxleft, frame)
#     drawbox(bboxright, frame)

#     drawc(frame, centleft)
#     drawc(frame, centright)

# #     write('')

#     cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK,2);
# #     cv2.putText(frame, "FPSd : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);
#     cv2.imshow("Tracking", frame)


#     if cv2.waitKey(2) == 13: #13 is the Enter Key
#         break
    
# releaseAll()
    
# cv2.destroyAllWindows()
# # fvs.stop()

In [251]:
cv2.destroyAllWindows()
fvs.stop()

In [10]:
def write(s, val, frame):
    cv2.putText(frame, s, (100, 50+val), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2);